<a href="https://colab.research.google.com/github/alyilmaz99/surface_crack_cnn_detect/blob/main/surface_crack_cnn_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from zipfile import ZipFile

from PIL import Image
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.preprocessing import image
##https://www.kaggle.com/datasets/arunrk7/surface-crack-detection

In [ ]:
! pip install -q kaggle

In [ ]:
!pip install -q opendatasets
import opendatasets as od

# od.download('https://www.kaggle.com/datasets/arunrk7/surface-crack-detection')



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alyyilmaz
Your Kaggle Key: ··········


100%|██████████| 233M/233M [00:02<00:00, 89.0MB/s]


In [ ]:
data_folder = ''

infected_folder = os.path.join(data_folder, '/content/surface-crack-detection/Positive')
notinfected_folder = os.path.join(data_folder, '/content/surface-crack-detection/Negative')
images = []
labels = []


for filename in os.listdir(infected_folder):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(infected_folder, filename))
        img = img.resize((224, 224))
        img = np.array(img)
        if len(img.shape) == 3:
            images.append(img / 255.0)
            labels.append(1)
for filename in os.listdir(notinfected_folder):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(notinfected_folder, filename))
        img = img.resize((224, 224))
        img = np.array(img)
        if len(img.shape) == 3:
            images.append(img / 255.0)
            labels.append(0)


In [ ]:
images = np.array(images)
labels = np.array(labels)

x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train, y_train = shuffle(x_train, y_train, random_state=42)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
notinfected_indices = [i for i, label in enumerate(labels) if label == 0]

for i in range(5):
    plt.subplot(3, 5, i + 1)
    plt.imshow(images[notinfected_indices[i]])
    plt.title("Notinfected")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:

model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Dropout(0.6))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.6))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(2))

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping]
)

In [ ]:
test_image = Image.open('data/test/img7.jpg')
test_image = test_image.resize((224, 224))
test_image = np.array(test_image) / 255.0


predictions = model.predict(np.expand_dims(test_image, axis=0))

predicted_class = np.argmax(predictions, axis=1)

if predicted_class == 0:
    predict = "Notifected"

else:
   predict = "Infected"

plt.imshow(test_image)
plt.title( predict)
plt.axis('off')
plt.show()